<a href="https://colab.research.google.com/github/k650250/estatapi/blob/main/sample_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# estatapiモジュールの使用例（Google Colaboratory版）

冒頭の「Open in Colab」をクリックし、「ドライブにコピー」コマンドをクリックしてご使用ください。


## 「アプリケーションID」と「API リクエスト URL」を指定

※[e-Statのユーザ登録](https://www.e-stat.go.jp/mypage/user/preregister)を行い、マイページにログインし、API機能(アプリケーションID発行)から、アプリケーションIDを取得してください。

In [ ]:
# アプリケーションID
appId = 'ここに取得したアプリケーションIDを貼り付けてください'

# API リクエスト URL - 平成22年国勢調査 https://www.e-stat.go.jp/stat-search/database?statdisp_id=0003410439
url = 'http://api.e-stat.go.jp/rest/3.0/app/getStatsData?appId=&lang=J&statsDataId=0003410439&metaGetFlg=Y&cntGetFlg=N&explanationGetFlg=Y&annotationGetFlg=Y&sectionHeaderFlg=1'

## モジュール（estatapi.py）をダウンロード

In [ ]:
!wget --no-check-certificate https://raw.githubusercontent.com/k650250/estatapi/main/estatapi.py

## モジュール／ライブラリを読み込む

In [ ]:
import estatapi
import numpy as np
import pandas as pd

## APIを介してデータを取得

In [ ]:
df = estatapi.StatsDataForPandas(url, appId).to_df()

df

## 必要なデータを選択

In [ ]:
df = df.loc[(df["時間軸（調査年）"] == "2010年") & (df["@unit"] == "人") & (
    (df["年齢（各歳）年齢不詳，再掲有り2010"] == "総数") |
    (df["年齢（各歳）年齢不詳，再掲有り2010"] == "０歳") |
    (df["年齢（各歳）年齢不詳，再掲有り2010"] == "１歳") |
    (df["年齢（各歳）年齢不詳，再掲有り2010"] == "２歳")), :].astype({"$": np.int})

df

## 目的の統計表を表示

In [ ]:
df = pd.pivot_table(data=df,
                    index="男女別2010",
                    columns="年齢（各歳）年齢不詳，再掲有り2010",
                    values="$").loc[("総数（男女別）", "男", "女"),
                                    ("総数", "０歳", "１歳", "２歳")]
df.index = ("全国", "男", "女")
df.columns = ("総数（年齢）", "０歳", "１歳", "２歳")
with pd.option_context('display.float_format', "{:,.0f}".format):
    display(df.astype(np.float))